<a href="https://colab.research.google.com/github/jiveshkalra/NavDrishtiAI-/blob/main/NavDrishtiAI_Testing_1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Dependencies

In [ ]:
!pip install transformers timm einops -q
!pip install flash-attn -q
!pip install pyngrok -q
!pip install -U openai-whisper -q
!pip install -U pip setuptools wheel -q
!pip install Flask-Cors -q
!pip install -U 'spacy[cuda12x]' -q
!pip install cupy-cuda12x -q
!python -m spacy download en_core_web_md
!pip install easyocr -q

In [ ]:
!pip install soundfile -q

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
from IPython.display import Audio, display
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image


## Loading the Model

In [ ]:
model_id = "vikhyatk/moondream2"
revision = "2024-03-13"
moondream_model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision
).to("cuda")
moondream_tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

## For Running It on Colab

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='frame.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Wait for Capture to be clicked.
      await new Promise((resolve) => setTimeout(resolve,5000));

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

## For Flask Application

In [ ]:
!ngrok config add-authtoken 2T3XptWm61TCFJwaBLgyUNo2RKk_2GAtdhbrNRjkKy9drqdMN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
def describe_image(image_location ,prompt= "Tell me what is in front of me in detail"):
  image = Image.open(image_location)
  enc_image = moondream_model.encode_image(image)
  return moondream_model.answer_question(enc_image,prompt, moondream_tokenizer)

# Loading Models
import whisper
import spacy
import easyocr
whisper_model = whisper.load_model("medium.en")
nlp = spacy.load("en_core_web_md")
reader = easyocr.Reader(['en'],gpu=True)

In [ ]:
!rm -r /content/uploads

In [ ]:
def run_ocr(image_path):
  results =reader.readtext(image_path,paragraph=True,text_threshold=0.65,low_text =0.5,adjust_contrast =0.65)
  ending_texts = []
  for result in results:
    bbox = result[0]
    ending_texts.append(result[1])
  return " ".join(ending_texts).lower()

In [ ]:
import time
import datetime
from flask import Flask, request, jsonify
import base64
import os
from pyngrok import ngrok, conf
from flask_cors import CORS
import subprocess
import threading
from PIL import Image
import soundfile as sf
from werkzeug.utils import secure_filename


UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

app = Flask(__name__)
port = 5000
CORS(app)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def mode_switch(query_mode,mode):
  print(f'mode ->{mode}')
  print(f'query_mode ->{query_mode}')
  if query_mode == "introduction":
    to_play = '''
    Welcome to NaavDrishti AI . NaavDrishtiAI is a groundbreaking Aritificial Intelligence powered solution designed to empower the visually impaired community and enhance their quality of life.
    Currently It has Three Modes which you can toggle between and each mode has separate functionality
    Firstly is the auto mode , in this the AI automatically takes picture of your surroundings in every 20 seconds and analyses your surroundings for any percieved threats and gives you constant updates on whats happening arround you.
    Secondly is the question mode , this is toggled on by default and in this you can ask the AI model any question while holding the button on the bottom of your screen , and the AI will analyse the image feed from your camera and answer your queries accordingly.
    and Thirdly is the read mode ,in this you can ask the AI to read the text out of a book , newspaper or any other document.

    To switch between different modes, Say Turn On Mode_name  mode , for example : turn on auto mode.
    '''
    return 'question',to_play
  if query_mode == mode:
    to_play= f"{mode} is already turned on.Please say some other mode to switch."
  else:
    to_play= f'Turning on {query_mode} mode.'
    mode = query_mode
  return mode,to_play

@app.route('/audio-processing', methods=['POST'])
def upload_audio():
  app.logger.info("Received audio upload request")
  to_play="Something went wrong ,Please try again."

  audio_blob = request.files['audioBlob']
  image_data = request.form['image']
  mode = request.form['mode']

  with open(f'{UPLOAD_FOLDER}/audio.wav', 'wb') as f:
    f.write(audio_blob.read())
  result = whisper_model.transcribe(f"{UPLOAD_FOLDER}/audio.wav")
  query = result["text"]
  query_cmd = nlp(query.lower().replace('turn','').replace(' on','').replace('.','').replace('mode','').replace('give','').replace('me','').strip())
  question_cmd = nlp('question')
  auto_cmd = nlp('auto')
  read_cmd = nlp('read')
  introduction_cmd = nlp('introduction')
  question_similarity = query_cmd.similarity(question_cmd)
  auto_similarity = query_cmd.similarity(auto_cmd)
  read_similarity = query_cmd.similarity(read_cmd)
  introduction_similarity = query_cmd.similarity(introduction_cmd)
  similarities = {
      "question": question_similarity,
      "auto": auto_similarity,
      "read": read_similarity,
      "introduction": introduction_similarity,
  }
  highest_similarity_variable = max(similarities, key=similarities.get)

  highest_similarity_value = similarities[highest_similarity_variable]
  print(highest_similarity_variable)
  if highest_similarity_value > 0.65:
    query_mode = highest_similarity_variable
    mode,to_play = mode_switch(query_mode,mode)
  else:
    try:
      image_path = download_image(image_data)
    except Exception as e:
      to_play = "Sorry , An error occured, please try again later."
    if mode == 'question':
      try:
        to_play = describe_image(image_path,query)
      except Exception as e:
        to_play = "Sorry , An error occured in running the ai interface, please try again later."
    elif mode == 'read':
      try:
        to_play = run_ocr(image_path)
      except Exception as e:
        to_play = "Sorry , An error occured in running the ai interface, please try again later."
    elif mode == 'auto':
      to_play= f'You are currently in Automatic Summary Mode,If you are trying to ask a question , please say TURN ON QUESTION MODE to toggle question mode and then ask queries'
    else:
      to_play= "Unable to understand the query ,please try again"

  return jsonify({"text":query,"query_cmd":str(query.lower().replace('.','').replace('turn','').replace( ' on','').replace('mode','').strip()),"to_play":to_play,"mode":mode,"highest_similarity_variable":highest_similarity_variable,"highest_similarity_value":highest_similarity_value});

def download_image(image_data):
  image_format = image_data.split(';')[0].split('/')[1]
  decoded_data = base64.b64decode(image_data.split(',')[1])
  time_rn = datetime.datetime.now().strftime("%f")

  filename = f'{time_rn}.{image_format}'
  image_path = os.path.join(UPLOAD_FOLDER, filename)

  with open(image_path, 'wb') as f:
      f.write(decoded_data)
  return image_path

@app.route('/process_image', methods=['POST'])
def process_image_api():
  if request.method == 'POST':
    try:
      if 'image' not in request.form or 'prompt' not in request.form:
        return jsonify({'error': 'Missing image data or prompt in request'}), 400

      image_data = request.form['image']
      prompt = request.form['prompt']
      mode = request.form['mode']
      threat_prompt = 'is there any threat in this image?'
      try:
        image_path = download_image(image_data)
      except Exception as e:
        return jsonify({'to_play': f'Error in downloading the image: {e}'}), 200
      try:
        answers = moondream_model.batch_answer(
            images=[Image.open(image_path), Image.open(image_path)],
            prompts=[prompt,threat_prompt],
            tokenizer=moondream_tokenizer,
        )
        description = answers[0]
        threat_detection = answers[1]
        print(threat_detection)
        if threat_detection.split(',')[0].lower().strip()=='no':
          return jsonify({"to_play":description}),200
        elif threat_detection.split(',')[0].lower().strip()=='yes':
          threat_description ="".join(threat_detection.split(',')[1:])
          to_play = f"ALERT : Threat Detected :{threat_description}"
          return jsonify({"to_play":to_play}),200
        else:
          return jsonify({"to_play":description}),200


      except Exception as e:
        return jsonify({'to_play': f'Error running ai : {e}'}), 200
    except Exception as e:
      print("Error processing image:", e)
      return jsonify({'to_play': 'Error processing image'}), 200

@app.route('/',methods=["GET"])
def index():
  return "Hello World"

def start_ngrok():
    subprocess.Popen(['ngrok', 'http', f'--domain=herring-notable-physically.ngrok-free.app' ,str(port)])

    # public_url = ngrok.connect(port).public_url
    # print(f" * ngrok tunnel {public_url} -> http://127.0.0.1:{port}")


if __name__ == '__main__':
    start_ngrok()
    app.run(host='127.0.0.1', port=port, use_reloader=False, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Mar/2024 11:14:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Mar/2024 11:15:12] "GET / HTTP/1.1" 200 -
INFO:__main__:Received audio upload request


question


INFO:werkzeug:127.0.0.1 - - [19/Mar/2024 11:15:37] "POST /audio-processing HTTP/1.1" 200 -
INFO:__main__:Received audio upload request
INFO:werkzeug:127.0.0.1 - - [19/Mar/2024 11:15:48] "POST /audio-processing HTTP/1.1" 200 -


question
mode ->question
query_mode ->question


...

Yes, there is a threat in this image. The image features a large brown and black snake, possibly a cobra, coiled up and sitting in the jungle. Snakes, especially cobras, are known to be dangerous predators that can pose a threat to humans and other animals. The presence of the snake in the jungle indicates that it might be hunting for prey or protecting its territory, which can be a potential threat to any nearby humans or animals.


INFO:werkzeug:127.0.0.1 - - [19/Mar/2024 11:34:11] "POST /process_image HTTP/1.1" 200 -


In the image, there is no immediate threat to the lion as it is lying down in a grassy field. However, the presence of a mountain in the background could potentially pose a threat if the lion were to venture too close to it. While the lion is not actively hunting or showing signs of aggression, it is always important to be aware of the surroundings and potential dangers when in the wild, even for animals like lions.


INFO:werkzeug:127.0.0.1 - - [19/Mar/2024 11:34:41] "POST /process_image HTTP/1.1" 200 -


In the image, there is no immediate threat to the lion as it is lying down in a grassy field. However, the presence of a mountain in the background could potentially pose a threat if the lion were to venture too close to it. While the lion is not actively hunting or showing signs of aggression, it is always important to be aware of the surroundings and potential dangers when in the wild, even for animals like lions.


## Extra Mess


In [ ]:
from transformers import DPTImageProcessor, DPTForDepthEstimation
import torch
import numpy as np
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

processor = DPTImageProcessor.from_pretrained("Intel/dpt-large")
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large").to('cuda')

# prepare image for the model
inputs = processor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    predicted_depth = outputs.predicted_depth

# interpolate to original size
prediction = torch.nn.functional.interpolate(
    predicted_depth.unsqueeze(1),
    size=image.size[::-1],
    mode="bicubic",
    align_corners=False,
)

# visualize the prediction
output = prediction.squeeze().cpu().numpy()
formatted = (output * 255 / np.max(output)).astype("uint8")
depth = Image.fromarray(formatted)


preprocessor_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor